In [ ]:
!pip install transformers
!pip install tokenizers
!pip install datasets

In [ ]:
from datasets import load_dataset
wikiann_dataset = load_dataset("wikiann","id")

In [ ]:
from datasets import load_dataset
indonlu_dataset = load_dataset("indonlu","nergrit")

In [ ]:
wikiann_label_names=dataset["train"].features["ner-_tags"].feature_names

In [ ]:
indonlu_dataset.DatasetInfo()

In [ ]:
indonlu_dataset=